In [15]:
import weaviate
import json

import os
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index import VectorStoreIndex, ServiceContext
from llama_index import SimpleDirectoryReader
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index.vector_stores import WeaviateVectorStore
from llama_index import VectorStoreIndex, StorageContext
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
from llama_index.response.notebook_utils import display_response

In [3]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    # model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

total size (MB): 4368.44


4167it [06:26, 10.79it/s]                          
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_

In [6]:
documents = SimpleDirectoryReader(
    input_files = ["/home/kganapa/ResearchLLM/data/HGNN_General_Hypergraph_Neural_Networks.pdf"]
).load_data()

documents = Document(text = "\n\n".join([doc.text for doc in documents]))

In [7]:
def get_build_index(documents,llm,embed_model="local:/home/kganapa/ResearchLLM/UAE-Large-V1",sentence_window_size=3,save_dir="./vector_store/index"):
  
  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

  sentence_context = ServiceContext.from_defaults(
      llm = llm,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  if not os.path.exists(save_dir):
        # create and load the index
        index = VectorStoreIndex.from_documents(
            [documents], service_context=sentence_context
        )
        index.storage_context.persist(persist_dir=save_dir)
  else:
      # load the existing index
      index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
      )

  return index

In [8]:
# get the vector index
vector_index = get_build_index(documents=documents, llm=llm, embed_model="local:/home/kganapa/ResearchLLM/UAE-Large-V1", sentence_window_size=3, save_dir="./vector_store/index")

/home/kganapa/ResearchLLM/resllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )
  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine

In [12]:
query_engine = get_query_engine(sentence_index=vector_index, similarity_top_k=6, rerank_top_n=2)

config.json: 100%|██████████| 799/799 [00:00<00:00, 6.65MB/s]
model.safetensors: 100%|██████████| 1.11G/1.11G [01:37<00:00, 11.5MB/s]
tokenizer_config.json: 100%|██████████| 443/443 [00:00<00:00, 3.75MB/s]
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:00<00:00, 11.7MB/s]
tokenizer.json: 100%|██████████| 17.1M/17.1M [00:01<00:00, 11.7MB/s]
special_tokens_map.json: 100%|██████████| 279/279 [00:00<00:00, 3.04MB/s]


In [16]:
while True:
  query="What are the hypergraph convolutions mentioned?"
  response = query_engine.query(query)
  display_response(response)
  print("\n")

Llama.generate: prefix-match hit

llama_print_timings:        load time =   34857.19 ms
llama_print_timings:      sample time =      34.48 ms /   145 runs   (    0.24 ms per token,  4205.58 tokens per second)
llama_print_timings: prompt eval time =   53553.77 ms /   614 tokens (   87.22 ms per token,    11.47 tokens per second)
llama_print_timings:        eval time =   13384.07 ms /   144 runs   (   92.94 ms per token,    10.76 tokens per second)
llama_print_timings:       total time =   67238.52 ms /   758 tokens


**`Final Response:`** The hypergraph convolutions mentioned in the context are two specific types of convolution layers used in Hypergraph Neural Networks (HGNNs). The first one is called HGNNConv, which is defined from the spectral aspect. The second one is called HGNNConv\_spatial, which is defined from the spatial aspect. The spectral aspect refers to the use of graph spectral theory to define the convolution operation, while the spatial aspect refers to the use of the spatial structure of the hypergraph to define the convolution operation. The text describes the spatial aspect of the hypergraph convolution in more detail, specifically a two-stage message passing framework for spatial hypergraph convolution.

Llama.generate: prefix-match hit



llama_print_timings:        load time =   34857.19 ms
llama_print_timings:      sample time =      45.58 ms /   184 runs   (    0.25 ms per token,  4036.68 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   17126.38 ms /   184 runs   (   93.08 ms per token,    10.74 tokens per second)
llama_print_timings:       total time =   17527.62 ms /   185 tokens


**`Final Response:`** The hypergraph convolutions mentioned in the context are two specific types of convolution layers used in Hypergraph Neural Networks (HGNNs). The first one is called HGNNConv, which was proposed in a conference version. The second one is called HGNNConv\_spatial, which was proposed in the journal version. The spatial hypergraph convolution layer is a general framework for spatial convolution on hypergraphs, which is used in both HGNNConv and HGNNConv\_spatial. The spatial convolution on hypergraphs is defined as the aggregation of neighbor vertex messages to update each vertex according to the hyperpath between the central vertex and each vertex in its neighborhood. The hypergraph incidence matrix and hyperedge weight matrix are generated from the constructed hyperedge groups and fed into the Hypergraph Convolution Layer for further computation.

...

Llama.generate: prefix-match hit



llama_print_timings:        load time =   34857.19 ms
llama_print_timings:      sample time =      36.99 ms /   146 runs   (    0.25 ms per token,  3946.59 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   13677.94 ms /   146 runs   (   93.68 ms per token,    10.67 tokens per second)
llama_print_timings:       total time =   14006.49 ms /   147 tokens


**`Final Response:`** The hypergraph convolutions mentioned in the context are two specific types of convolution layers used in Hypergraph Neural Networks (HGNNs). The first one is called HGNNConv, which was proposed in a conference version. The second one is called HGNNConv\_spatial, which was proposed in the journal version. The spatial hypergraph convolution layer is a general framework for spatial convolution on hypergraphs, which is used in both HGNNConv and HGNNConv\_spatial. The spatial convolution on hypergraphs is a method for aggregating messages from neighbor vertices to update the central vertex according to the path between them in the hypergraph.

Llama.generate: prefix-match hit


KeyboardInterrupt: 